In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.4 범위내 추출

In [31]:
import pandas as pd

# 파일 경로 설정
file_path_1 = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED.csv'


file_path_2 = '/content/drive/MyDrive/Kwargs/031. 부정변환/Generated_Combinations.csv'

# 첫 번째 데이터셋 불러오기
df1 = pd.read_csv(file_path_1)
df1['company'] = df1['company'].replace('포스코 홀딩스', 'POSCO홀딩스')


# 첫 번째 데이터셋의 라벨별 데이터 수 출력
print("First dataset (A3_remaster_combined_ANONYMIZED) label distribution:\n", df1['positivity'].value_counts())

First dataset (A3_remaster_combined_ANONYMIZED) label distribution:
 positivity
 1.0    4599
 0.0    1277
-1.0     751
Name: count, dtype: int64


In [32]:
df1['company'].value_counts()

,count
company,
현대차,954
현대모비스,729
삼성SDI,701
삼성물산,519
KB금융,479
기아,475
SK하이닉스,472
POSCO홀딩스,447
신한지주,402


In [33]:
# 두 번째 작업: -1 라벨 보충하기
df2 = pd.read_csv(file_path_2)

# 모든 데이터에 라벨 -1 부여
df2['positivity'] = -1

df2 = df2.rename(columns={'text': 'full_text'})


In [47]:
# df1의 라벨 분포 확인
label_counts_df1 = df1['positivity'].value_counts()

# df1에서 -1 라벨이 부족한 수 계산
desired_label_minus_1_count = 4599  # 목표 -1 라벨 개수
current_label_minus_1_count = label_counts_df1.get(-1, 0)
num_to_add = desired_label_minus_1_count - current_label_minus_1_count

# df2에서 num_to_add만큼 샘플링하여 df1에 추가
if num_to_add > 0:
    df2_sampled = df2.sample(num_to_add, random_state=42)

    # df1에 보충된 데이터 추가
    df1 = pd.concat([df1, df2_sampled], ignore_index=True)

# df1의 새로운 라벨 분포 확인
new_label_counts_df1 = df1['positivity'].value_counts()
new_label_counts_df1

,count
positivity,
-1.0,4599
1.0,4599
0.0,1277


In [48]:
# 각 텍스트의 길이를 계산하여 새로운 컬럼 'text_length' 추가
df1['text_length'] = df1['full_text'].apply(len)

# 라벨별로 텍스트의 평균 길이를 계산
average_text_length_per_label = df1.groupby('positivity')['text_length'].mean()
average_text_length_per_label

,text_length
positivity,
-1.0,1320.856491
0.0,1409.266249
1.0,1366.861274


In [57]:
df1.shape

(10475, 11)

In [49]:
import pandas as pd

# 텍스트에서 마침표를 기준으로 특정 길이 이후의 텍스트를 제거하는 함수
def truncate_text_at_period(text, start=1300, end=1500, step=100):
    # 시작 구간부터 끝 구간까지 마침표를 찾는다
    while start <= len(text):
        # 주어진 범위 내에서 마침표 찾기
        period_pos = text.find('.', start, min(end, len(text)))

        # 마침표가 해당 범위에 있으면 그 이후의 텍스트를 제거
        if period_pos != -1:
            return text[:period_pos + 1]  # 마침표까지 포함하여 자르기

        # 마침표가 없으면 다음 100자 범위를 설정
        start = end
        end += step

    # 지정된 범위 내에서 마침표를 찾지 못하면 원본 텍스트 반환
    return text

# 가상의 df1 데이터 예시 - 실제 데이터셋을 불러오세요
# df1 = pd.read_csv("your_file.csv")

# 'positivity'가 -1인 텍스트들만 필터링하여 복사
df_minus_0_1 = df1[df1['positivity'] == -1].copy()

# 텍스트 수정: 각 텍스트에서 마침표를 찾아 그 이후 텍스트 제거
df_minus_0_1['full_text'] = df_minus_0_1['full_text'].apply(lambda x: truncate_text_at_period(x, 1300, 1400, 100))

# 원본 데이터프레임에 수정된 텍스트 반영
df1.loc[df1['positivity'] == -1, 'full_text'] = df_minus_0_1['full_text']

# 수정된 결과 확인
print("수정된 텍스트 샘플 확인:")
print(df1[df1['positivity'] == -1][['full_text']].head())

# 텍스트 길이 계산을 위한 'text_length' 컬럼 추가
df1['text_length'] = df1['full_text'].apply(len)

# 라벨별로 텍스트의 평균 길이를 계산
average_text_length_per_label = df1.groupby('positivity')['text_length'].mean()

# 평균 텍스트 길이 출력
print("라벨별 평균 텍스트 길이:")
print(average_text_length_per_label)


수정된 텍스트 샘플 확인:
                                            full_text
1   '보장성 실적 반 토막' 한화생명 체질 개선 '먹구름' 자본 여력이 계속 생명보험업...
2   오너일가 지분 많은 기업 기피하는 애널리스트들 하지만 일감 몰아주기 제한 등 나날이...
4   이슈분석]대형IT서비스 3사, 내부거래 얼마나 했나 2017년 국내 IT서비스 시장...
8   수익 낮은 퇴직연금, 수수료 다 받아서 되겠나 대로 전면개편 나서신한금융그룹이 퇴직...
13  보험사 퇴직연금 성적 까보니… ‘집토끼’만 가득 이에 반해 보험사는 아직까지 별다른...
라벨별 평균 텍스트 길이:
positivity
-1.0    1320.856491
 0.0    1409.266249
 1.0    1366.861274
Name: text_length, dtype: float64


In [50]:
# 파일 경로 설정
# file_path = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_mapped.csv'
# df1.to_csv(file_path, index=False)

In [51]:
# Load company names from CSV if needed
company_names_df = pd.read_csv('/content/drive/MyDrive/Kwargs/companynames.csv',encoding='euc-kr')
company_names_df = company_names_df.dropna()
all_companies = company_names_df['종목명'].tolist()

In [52]:
# 기업명 통일 매핑 (기존 딕셔너리)
company_name_mapping = {
    '삼성전자': ['Samsung Electronics', '삼성전자', '삼전'],
    'POSCO홀딩스': ['POSCO홀딩스', '포스코홀딩스','포스코 홀딩스', '포스코그룹', 'POSCO', '포스코'],
    '현대백화점': ['Hyundai Department Store', '현대백화점', 'HDC', '현백'],
    'SK이노베이션': ['에스케이이노베이션', 'SK Innovation', 'SK이노베이션', 'SKI'],
    '대한항공': ['Korean Air', '대한항공', 'KAL'],
    '삼성물산': ['Samsung C&T', '삼성물산', '삼성건설',],
    '현대차': ['Hyundai Motor', '현대자동차', 'Hyundai', '현대차', '현대車', 'HMC'],
    '금호석유': ['금호석유화학', 'Kumho Petrochemical', '금호석유', '금호화학', '금호석화'],
    '카카오': ['다음카카오', 'KAKAO', 'Kakao', '카카오'],
    '효성': ['Hyosung', '효성그룹', '효성'],
    '동국제강': ['Dongkuk Steel', 'DONGKUK', '동국제강'],
    '하림지주': ['Harim Holdings', '하림지주', '하림그룹', '하림'],
    'CJ제일제당': ['CJ CheilJedang', 'CJ제일제당', 'CJ그룹', 'CJ', '씨제이제일제당','씨제이', '제일제당'],
    'LG에너지솔루션': ['LG Energy Solution', 'LG에너지솔루션', 'LG에솔', 'LGES'],
    '삼성바이오로직스': ['Samsung Biologics', '삼성바이오로직스', '삼성바이오','삼바'],
    '셀트리온': ['CELLTRION', 'Celltrion', '셀트리온'],
    '하나금융지주': ['Hana Financial Group', '하나금융지주', '하나금융', '하나은행'],
    'NAVER': ['네이버주식회사', 'NAVER', '네이버'],
    'LG전자': ['LG Electronics', 'LG전자', 'LGE', '엘지전자'],
    '한화에어로스페이스': ['Hanwha Aerospace', '한화에어로스페이스', '한화에어로'],
    'GS에너지': ['GS Energy', 'GS에너지', 'GS 에너지'],
    '우리금융지주': ['Woori Financial Group', '우리금융지주', '우리금융', '우리은행'],
    '카카오뱅크': ['KakaoBank', '카카오뱅크', '카뱅'],
    'KB금융': ['KB Kookmin Bank', 'KB금융지주', 'KB국민은행', 'KB금융', '국민은행'],
    '한화': ['Hanwha', '한화그룹', '한화지주', '한화'],
    '삼성중공업': ['Samsung Heavy Industries', '삼성중공업', 'SHI'],
    '기아': ['기아자동차', 'Kia Motors', 'Kia', '기아차', '기아'],
    '크래프톤': ['KRAFTON', 'Krafton', '크래프톤'],
    'LG화학': ['엘지화학', 'LG Chem', 'LG화학'],
    '삼성SDS': ['삼성에스디에스', 'Samsung SDS', '삼성SDS'],
    'SK텔레콤': ['SK Telecom', 'SK텔레콤','에스케이텔레콤', 'SKT'],
    'SK바이오팜': ['SK Biopharmaceuticals', 'SK BioPharm', 'SK바이오팜'],
    '아모레퍼시픽': ['Amorepacific', '아모레퍼시픽', '아모레'],
    '한국타이어앤테크놀로지': ['한국타이어앤테크놀로지', 'Hankook Tire', '한국타이어'],
    'LG생활건강': ['엘지생활건강', 'LG생활건강', 'LG H&H'],
    'SK하이닉스': ['에스케이하이닉스','SK하이닉스', 'SK hynix', '하이닉스'],
    '한온시스템': ['Hanon Systems', '한온시스템', '한온'],
    'LG유플러스': ['LG유플러스', 'LG U+', '엘지유플러스'],
    '현대모비스': ['Hyundai Mobis', '현대모비스', '모비스'],
    'LG디스플레이': ['LG Display', '엘지디스플레이','LG디스플레이', 'LGD'],
    '롯데지주': ['Lotte Corporation', '롯데지주'],
    'GS리테일': ['GS Retail', 'GS리테일', 'GS25' ],
    '넷마블': ['Netmarble', '넷마블'],
    'SK바이오사이언스': ['SK Bioscience', 'SK바이오사이언스', 'SK바사'],
    '롯데쇼핑': ['Lotte Shopping', '롯데쇼핑'],
    'GS': ['GS그룹', 'GS지주' 'GS'],
    'BGF리테일': ['BGF Retail', 'BGF리테일', 'BGF'],
    '한미약품': ['Hanmi Pharmaceutical', '한미약품'],
    'SK네트웍스': ['SK Networks', 'SK네트웍스'],
    '삼성전기': ['Samsung Electro-Mechanics', '삼성전기'],
    'SK아이이테크놀로지': ['SK IE Technology', 'SK아이이테크놀로지', 'SKIET'],
    'HMM': ['Hyundai Merchant Marine', '현대상선', 'HMM'],
    'KT': ['Korea Telecom', 'KT', '케이티'],
    '코웨이': ['Coway', '코웨이'],
    '팬오션': ['Pan Ocean', '팬오션'],
    '삼성SDI': ['삼성에스디아이', 'Samsung SDI', '삼성SDI'],
    '신한지주': ['Shinhan Financial Group', '신한금융지주', '신한금융', '신한'],
    '한국가스공사': ['한국가스공사', 'KOGAS', '가스공사'],
    '농심': ['Nongshim', '농심'],
    '영원무역': ['Youngone', '영원무역'],
    '금호타이어': ['Kumho Tire', '금호타이어'],
    '이마트': ['E-mart', '이마트'],
    '삼성생명': ['삼성생명보험', '삼성생명금융서비스', '삼성생명', '삼생'],
    '고려아연': ['Korea Zinc', '고려아연', 'KZ'],
    'S-Oil': ['에쓰오일', 'S-Oil', '에스오일', 'S-오일'],
    '엔씨소프트': ['엔씨소프트', 'NCsoft', 'NC소프트', 'NC'],
    '카카오페이': ['KakaoPay', '카카오페이', '카페이']
}

In [60]:
import pandas as pd
import re
import unicodedata
from tqdm import tqdm

##### 밸런싱한 데이터 이어서 사용
df = df1.copy()

# 유니코드 정규화 함수 (비문자형 데이터는 빈 문자열로 변환)
def normalize_text(text):
    """텍스트를 유니코드 NFC 형식으로 정규화, 비문자형 데이터는 빈 문자열로 변환"""
    if isinstance(text, str):
        return unicodedata.normalize('NFC', text)
    else:
        return ''  # NaN 값을 빈 문자열로 처리

# 기업명 매핑을 적용하여 표준화된 이름으로 정리하는 함수
def clean_company_name(name):
    """기업명 정리 및 표준화 (기업명 매핑 딕셔너리 적용)"""
    cleaned_name = normalize_text(name).strip()

    # 딕셔너리에서 매칭되는 이름 찾기 (가정: company_name_mapping이 존재)
    for standard_name, variations in company_name_mapping.items():
        if cleaned_name in variations:
            return standard_name

    # 매칭되는 것이 없으면 원래 이름 반환
    return cleaned_name

# 본문에서 기업명을 대체하는 함수
def preprocess_content(content):
    """본문 내용에서 기업명 대체 작업 (조사가 붙어도 매핑되도록 보완)"""
    for original_name, variations in sorted(company_name_mapping.items(), key=lambda x: -len(x[0])):
        if any(variation in content for variation in variations):
            try:
                # 본문에서 변형된 기업명을 표준화된 이름으로 대체
                content = re.sub(rf'({"|".join(map(re.escape, variations))})([가-힣]+)?',
                                 lambda m: original_name + (m.group(2) if m.group(2) else ''),
                                 content)
            except re.error as e:
                print(f"Regex error: {e}")
    return content

# 기업명 매핑을 데이터프레임의 'full_text'와 'company' 컬럼에 대해 수행
def map_company_names(df):
    """기업명 매핑 및 본문 내용 매핑 작업 수행"""
    tqdm_bar = tqdm(total=len(df), desc="매핑 진행 중")

    for i, row in df.iterrows():
        # 각 행에 대해 누락 여부 확인을 위한 디버깅 메시지
        if i % 500 == 0:
            print(f"Processing row {i}: {row['company']} - {row['full_text'][:100]}")

        # 'full_text' 컬럼에서 매핑
        content = row['full_text']
        if pd.isna(content):
            print(f"Warning: Row {i} has NaN content, skipping this row.")
            continue  # NaN 값을 가진 행은 건너뛰기

        content = normalize_text(content)
        content = preprocess_content(content)

        # 매핑된 본문을 업데이트
        df.loc[i, 'full_text'] = content

        # 'company' 컬럼에서 매핑
        company_name = row['company']
        if pd.isna(company_name):  # 회사 이름이 없는 경우에도 본문에서 매핑된 기업명을 찾음
            for standard_name, variations in company_name_mapping.items():
                if any(variation in content for variation in variations):
                    company_name = standard_name
                    break

        cleaned_name = clean_company_name(company_name)

        # 매핑된 기업명을 업데이트
        df.loc[i, 'company'] = cleaned_name

        # 첫 5개는 모두 출력, 그 이후에는 500개마다 진행 상황 출력
        if i < 5 or i % 500 == 0:
            print(f"Row {i} - Original Company: {company_name}, Mapped Company: {cleaned_name}")
            print(f"Row {i} - Original Full Text: {row['full_text'][:200]}")

        if i % 500 == 0:
            tqdm_bar.update(500)

    tqdm_bar.close()
    return df

# 기업명 매핑 정보 로드
company_names_df = pd.read_csv('/content/drive/MyDrive/Kwargs/companynames.csv', encoding='euc-kr')
company_names_df = company_names_df.dropna()
all_companies = company_names_df['종목명'].tolist()

# 데이터 크기 확인
print(f"Initial number of rows in df: {len(df)}")

# 기업명 및 본문 내용 매핑 수행
mapped_df = map_company_names(df)

# 데이터 크기 확인 (매핑 후)
print(f"Number of rows after mapping: {len(mapped_df)}")

# 매핑 결과 저장
mapped_output_file = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped.csv'
mapped_df.to_csv(mapped_output_file, encoding='utf-8', index=False)

print(f"매핑 작업이 완료되었습니다: {mapped_output_file}")


Initial number of rows in df: 10475


매핑 진행 중:   0%|          | 0/10475 [00:00<?, ?it/s]

Processing row 0: 삼성생명 - '빅3' 암보험금 민원 결론낸다…지급의견서 제출 20일 금융당국 및 보험업계에 따르면 삼성생명은 최근 2건의 암보험 민원에 대한 보험금 지급을 결정하고 금감원에 지급 의견서를 제출
Row 0 - Original Company: 삼성생명, Mapped Company: 삼성생명
Row 0 - Original Full Text: '빅3' 암보험금 민원 결론낸다…지급의견서 제출 20일 금융당국 및 보험업계에 따르면 삼성생명은 최근 2건의 암보험 민원에 대한 보험금 지급을 결정하고 금감원에 지급 의견서를 제출했다. 금감원이 지급 여부를 검토하라고 지시한 다른 민원 건에 대해서도 순차적으로 지급 의견서를 낼 계획이다. 삼성생명 관계자는 환자의 주치의 판단이 분쟁 조정에서 가장 중요한 기
Row 1 - Original Company: 삼성생명, Mapped Company: 삼성생명
Row 1 - Original Full Text: '보장성 실적 반 토막' 한화생명 체질 개선 '먹구름' 자본 여력이 계속 생명보험업계 평균을 밑돌고 있는 한화생명의 입장에서 상품 체질 개선은 점점 다급한 숙제가 돼 가고 있다. 2일 금융감독원에 따르면 자산 기준 국내 10대 생보사들이 지난해 1. 3분기 보장성 보험에서 거둔 초회보험료는 총 7326억원으로 전년 동기. 초회보험료는 고객이 보험에 가입하고
Row 2 - Original Company: 삼성생명, Mapped Company: 삼성생명
Row 2 - Original Full Text: 오너일가 지분 많은 기업 기피하는 애널리스트들 하지만 일감 몰아주기 제한 등 나날이 강화되는 공정거래법 규제 때문에 주가는 최근 1년 사이 고점대비 50. 가까이 떨어지며 부진한 상황이다. 외부 시선 때문에 신사업도 적극적으로 추진하지 못해 애널리스트들은 관리가 어렵다며 외면하고 있다. A사 IR 담당자는 수년전만 해도 10여명의 애널리스트와 소통해왔

매핑 진행 중:  10%|▉         | 1000/10475 [00:00<00:06, 1374.62it/s]

Processing row 500: 현대차 - 정규직 KF94, 비정규직 면마스크…마스크 차별에 운다 전국금속노조 산하 현대자동차 비정규직지회장 김현제씨는 5일 CBS노컷뉴스와의 통화에서 코로나19 사태 이후 회사로부터 마스크
Row 500 - Original Company: 현대차, Mapped Company: 현대차
Row 500 - Original Full Text: 정규직 KF94, 비정규직 면마스크…마스크 차별에 운다 전국금속노조 산하 현대자동차 비정규직지회장 김현제씨는 5일 CBS노컷뉴스와의 통화에서 코로나19 사태 이후 회사로부터 마스크 한 장 받지 못했다며 착잡한 심경을 밝혔다. 김씨에 따르면 지난 2일까지 현대차 울산공장 비정규직지회에 소속된 28개 하청업체 전체에서 비정규직 노동자 1200여명에게 코로나19


...

Processing row 9500: nan - 판교점 신규 브랜드 매장은 현대서울의 성공적인 모델과 달리 MZ세대를 위한 맞춤형 큐레이션 전략이 부족하고, 개인차 키holder 비스듬이, 전기자전거 등 기존 백화점에서 부족했던
Row 9500 - Original Company: nan, Mapped Company: 
Row 9500 - Original Full Text: 판교점 신규 브랜드 매장은 현대서울의 성공적인 모델과 달리 MZ세대를 위한 맞춤형 큐레이션 전략이 부족하고, 개인차 키holder 비스듬이, 전기자전거 등 기존 백화점에서 부족했던 프리미엄 라이프스타일 콘텐츠가 크게 부족한 점이 특징이다. 신고범위에서 고유국가/사업장 특성을 고려하지 않고 환경·사회적 요인과 같은 지속가능 성과지표 수집·평가 체계를 구현하지


매핑 진행 중: 10500it [00:12, 819.64it/s]                           

Processing row 10000: nan - ESG 관점에서 SK하이닉스의 수스트레스지역 수자원관리는 미흡한 실정으로 생산사업장이 위치한 지역의 수스트레스 수준을 효과적으로 평가하지 못해 수소소소 우려 해소에 적절한 대책이 
Row 10000 - Original Company: 카카오, Mapped Company: 카카오
Row 10000 - Original Full Text: ESG 관점에서 SK하이닉스의 수스트레스지역 수자원관리는 미흡한 실정으로 생산사업장이 위치한 지역의 수스트레스 수준을 효과적으로 평가하지 못해 수소소소 우려 해소에 적절한 대책이 부족하다. 더욱이 ESG 관점에서 직무 순환과 MBA 교육 같은 성장 기회를 제공하지 못해 핵심 인재의 발전을 소홀히 하고 있다. 이해관계자 범주를 기준으로 중요한 해결되지 않은 


매핑 진행 중: 10500it [00:13, 807.42it/s]


Number of rows after mapping: 10475
매핑 작업이 완료되었습니다: /content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped.csv


In [61]:
mapped_df.shape

(10475, 11)

In [62]:
import pandas as pd
import re
import unicodedata
import random
from tqdm import tqdm

# 유니코드 정규화 함수 (문자열이 아닌 값이 들어오면 빈 문자열 반환)
def normalize_text(text):
    """텍스트를 유니코드 NFC 형식으로 정규화"""
    if isinstance(text, str):
        return unicodedata.normalize('NFC', text)
    else:
        return ''  # 문자열이 아닌 경우 빈 문자열 반환

# 기업명 클리닝 함수 (최소한의 정규화만 적용)
def clean_company_name(name):
    """기업명 클리닝: 불필요한 패턴만 제거"""
    cleaned_name = normalize_text(name)  # 유니코드 정규화
    prefix_suffix_patterns = [r'주식회사', r'㈜']  # 괄호나 다른 패턴을 제거하지 않음
    for pattern in prefix_suffix_patterns:
        cleaned_name = re.sub(pattern, '', cleaned_name)
    return cleaned_name.strip()  # 앞뒤 공백 제거

# 부분적으로 포함된 기업명 제거 함수
def remove_contained_names(names):
    """부분적으로 포함되는 이름을 제거하여 최종 기업명 리스트 생성"""
    final_names = set(names)
    for name in names:
        for other in names:
            if name != other and name in other:
                final_names.discard(name)
                break
    return list(final_names)

# 기업명 검색용 정규식 (유연한 경계 및 조사 처리)
def search_company_names(content, all_companies):
    """기사 내용에서 기업명을 추출하는 정규식"""
    companies_in_article = []
    for company in all_companies:
        if company.strip():  # 공백이 아닌 경우만 처리
            # 기업명 뒤에 조사 또는 문장 부호가 붙는 경우도 매칭
            pattern = rf'\b{re.escape(company)}(?:[가-힣]*?)\b'
            if re.search(pattern, content):
                companies_in_article.append(company)

    return list(set(companies_in_article))  # 중복 제거

# 익명화 수행 함수
def anonymize_company_names(df, all_companies):
    alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    company_used_anonymous = {clean_company_name(company): set() for company in all_companies}
    global_used_anonymous = set()

    for i, row in tqdm(df.iterrows(), total=len(df), desc="익명화 진행 중"):
        content = normalize_text(str(row['full_text']))  # 기사 내용 정규화

        # company 컬럼에 있는 기업명 가져오기 (clean 처리)
        original_company_name = clean_company_name(str(row['company']))

        # 공백이 아닌 경우만 기업명 리스트에 포함
        if original_company_name.strip():
            companies_in_article = search_company_names(content, all_companies)

            if original_company_name not in companies_in_article:
                companies_in_article.append(original_company_name)

        else:
            companies_in_article = search_company_names(content, all_companies)

        company_to_anonymous = {}
        used_letters = set(global_used_anonymous)

        for company in companies_in_article:
            normalized_company = clean_company_name(company)

            # 빈 문자열, 숫자 또는 'nan' 값은 건너뜀
            if not normalized_company or normalized_company.isdigit() or normalized_company.lower() == 'nan':
                continue

            if normalized_company not in company_used_anonymous:
                company_used_anonymous[normalized_company] = set()

            if normalized_company not in company_to_anonymous:
                already_used = company_used_anonymous[normalized_company]
                available_letters = list(set(alphabet) - used_letters - already_used)

                if not available_letters:
                    available_letters = list(set(alphabet) - already_used)

                random_letter = random.choice(available_letters)
                anonymous_name = f'{random_letter}사'

                company_to_anonymous[normalized_company] = anonymous_name
                used_letters.add(random_letter)
                company_used_anonymous[normalized_company].add(anonymous_name)
                global_used_anonymous.add(anonymous_name)

        # 기사 내 기업명을 익명화된 이름으로 치환
        for company in companies_in_article:
            normalized_company = clean_company_name(company)

            # company_to_anonymous 딕셔너리에 해당 기업명이 없는 경우를 처리
            if normalized_company not in company_to_anonymous:
                continue  # 만약 익명화할 이름이 없다면 건너뜁니다

            # 기업명 뒤에 조사나 문장 성분이 붙어도 익명화 처리
            pattern = rf'(?<!\w){re.escape(company)}([가-힣]*)'
            content = re.sub(pattern, lambda m: company_to_anonymous[normalized_company] + m.group(1), content)

        # 익명화된 기사 내용 업데이트
        df.at[i, 'full_text'] = content

        # '기업명' 컬럼에 익명화된 이름을 저장 (원래 기업명 'company'의 익명화 이름)
        df.at[i, '기업명'] = company_to_anonymous.get(original_company_name, original_company_name)

        # 첫 5개는 모두 출력, 그 이후에는 400개마다 출력
        if i < 30 or i % 400 == 0:
            print(f"Row {i} 익명화 매핑: {company_to_anonymous}")
            print(f"Row {i} 익명화 후 - 기업명: {df.at[i, '기업명']}, full_text: {df.at[i, 'full_text'][:200]}")

    return df

# 기업명 정리 함수 (불필요한 중복 제거)
def load_and_clean_company_names(df):
    # 'companynames.csv' 파일에서 기업명 로드 (인코딩: euc-kr)
    company_names_df = pd.read_csv('/content/drive/MyDrive/Kwargs/companynames.csv', encoding='euc-kr')
    companynames_csv_list = company_names_df.iloc[:, 0].tolist()

    # df['company']에서 공백이 없는 기업명만 추가
    df_company_list = df['company'].dropna().unique().tolist()
    df_company_list = [clean_company_name(name) for name in df_company_list if name.strip()]  # 공백 제외

    # 두 리스트 병합 후 중복 제거
    all_company_names = list(set(companynames_csv_list + df_company_list))

    # 불필요한 중복 제거
    final_names = remove_contained_names(all_company_names)
    return final_names

# 파일 경로 설정
file_path = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped.csv'

# 파일 로드
df = pd.read_csv(file_path)
df = df.sample(frac=1).reset_index(drop=True)

# 기업명 리스트 불러오기
all_companies = load_and_clean_company_names(df)

# 익명화 수행
anonymized_df = anonymize_company_names(df, all_companies)

# 익명화된 데이터 저장
output_file = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped_ANONYMIZED.csv'
anonymized_df.to_csv(output_file, encoding='utf-8', index=False)

print(f"익명화된 데이터가 저장되었습니다: {output_file}")


익명화 진행 중:   0%|          | 1/10475 [00:01<4:31:39,  1.56s/it]

Row 0 익명화 매핑: {'현대차': 'O사'}
Row 0 익명화 후 - 기업명: O사, full_text: O사, 2026년까지 중형급 무인 화물운송기 개발한다 O사는 지난해 공개한 도심 항공 모빌리티. Urban Air Mobility. 의 상용화 계획을 바탕으로 UAM 기체 양산에 앞서 2026년까지 화물 운송용 Cargo UAS을 선보일 예정이다. 이를 통해 UAM 양산기술 노하우를 축적하는 한편. 무인 항공운송 산업 생태계를 조기에 조성하고 상업화를 주도


익명화 진행 중:   0%|          | 2/10475 [00:04<6:17:00,  2.16s/it]

Row 1 익명화 매핑: {'삼성전자': 'Z사'}
Row 1 익명화 후 - 기업명: Z사, full_text: 2030년까지 물 100% 환원·폐배터리 재활용… Z사 '탄소중립' 혁신기술 구체화 2050년까지 모든 플라스틱 부품에 재생레진을 적용하고. 수거한 모든 폐배터리에서 광물을 추출. 재활용해 신제품에 재활용한다는 계획이다. Z사는 지난 16일 서울 중구 태평로빌딩에서 . 신환경경영전략 간담회. 를 열고 기후위기 극복 등 지구환경 개선에 기여할 친환경 혁신기술


익명화 진행 중:   0%|          | 3/10475 [00:06<6:43:56,  2.31s/it]

...

Row 10000 익명화 매핑: {'KB금융': 'K사'}
Row 10000 익명화 후 - 기업명: K사, full_text: 윤종규 K사 회장이 '꿀벌'에 진심인 이유 GFANZ는 금융을 통해 넷제로 경제를 촉진하고 기후위기를 극복하기 위해 작년 4월 설립됐다. 탄소중립 추진을 위한 아태지역의 중요성을 감안해 싱가포르에 첫 아시아. 태평양 사무소를 개설하고 자문위원회를 구성했다. 싱가포르 중앙은행 총재. 엄우종 아시아개발은행. 이사장 등으로 구성돼 있다. 국내 금융기관에서는 유일


익명화 진행 중:  99%|█████████▉| 10401/10475 [5:14:33<02:05,  1.70s/it]

Row 10400 익명화 매핑: {}
Row 10400 익명화 후 - 기업명: nan, full_text: ESG 관점에서 현대해상은 평가 과정을 통해 조직의 성과에 중요한 주요 쟁점을 우선순위에 두고 있지 못하고, 검증팀은 이와 관련해 상당한 누락을 파악하였다. ESG 관점에서 보면 협동조합 협약에 따른 장애인 스포츠선수 취업기회 증진에 대한 의지 부족이 안타깝게 여겨졌고 중앙대 후원금은 2008년부터 두산그룹이 의도한 대로 학교 교육·시설비 투자에 효과적으로


익명화 진행 중: 100%|██████████| 10475/10475 [5:16:57<00:00,  1.82s/it]


익명화된 데이터가 저장되었습니다: /content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped_ANONYMIZED.csv


In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_remaster_COMBINED_mapped_ANONYMIZED.csv', encoding='utf-8')
df.shape

(10475, 12)

In [8]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# 1. 텍스트 분류용 데이터셋 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # -1, 0, 1 라벨을 0, 1, 2로 매핑 (라벨 값이 음수면 문제가 발생하므로)
        if label == -1:
            label = 0
        elif label == 0:
            label = 1
        elif label == 1:
            label = 2

        # 텍스트를 토크나이저로 처리
        encoding = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 2. KoELECTRA 모델 정의
class KoELECTRAClassifier(nn.Module):
    def __init__(self, model_name, num_labels=3):
        super(KoELECTRAClassifier, self).__init__()
        # KoELECTRA 모델 로드
        self.electra = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # KoELECTRA의 forward 함수 호출
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

# 3. 학습 루프 정의
def train_model(model, data_loader, optimizer, device, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        total_loss = 0.0
        progress_bar = tqdm(data_loader, desc=f"Epoch {epoch + 1}")

        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 모델의 출력을 얻고 손실 계산
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': total_loss / len(data_loader)})

        print(f"Epoch {epoch + 1} completed. Average Loss: {total_loss / len(data_loader)}")

# 4. 데이터셋 및 토크나이저 설정
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-small-discriminator')
max_len = 128

# 학습 데이터 로드
train_texts = df['full_text'].tolist()  # 텍스트 데이터
train_labels = df['positivity'].tolist()  # 라벨 데이터

# 데이터셋 및 데이터로더 생성
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len=max_len)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 5. 모델 및 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KoELECTRAClassifier('monologg/koelectra-small-discriminator').to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

# 6. 학습 실행
train_model(model, train_loader, optimizer, device, num_epochs=3)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Epoch 1: 100%|██████████| 655/655 [37:58<00:00,  3.48s/it, loss=0.58]


Epoch 1 completed. Average Loss: 0.5801050637969534
Epoch 2/3


Epoch 2: 100%|██████████| 655/655 [37:15<00:00,  3.41s/it, loss=0.347]


Epoch 2 completed. Average Loss: 0.34658306595933347
Epoch 3/3


Epoch 3: 100%|██████████| 655/655 [37:50<00:00,  3.47s/it, loss=0.286]


Epoch 3 completed. Average Loss: 0.2861586943695336
모델이 저장되었습니다.


In [21]:
# KoELECTRA 내부의 Electra 모델을 저장
for param in model.electra.parameters():
    if not param.is_contiguous():
        param.data = param.contiguous()

# 이제 모델을 저장합니다.
model.electra.save_pretrained('/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned')
tokenizer.save_pretrained('/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned')


('/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned/vocab.txt',
 '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned/added_tokens.json')

In [23]:
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# GPU 또는 CPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 저장된 KoELECTRA 모델과 토크나이저 로드
model_path = '/content/drive/MyDrive/Kwargs/023. esg 긍부정도 라벨러/업데이트/A3_KoELECTRA_finetuned'
model = ElectraForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = ElectraTokenizer.from_pretrained(model_path)

model.eval()  # 평가 모드로 전환


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [24]:
eval_df = df.copy()

In [25]:
from torch.utils.data import DataLoader

# 평가용 데이터셋 정의 (학습용 데이터셋과 동일한 형식)
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # -1, 0, 1 라벨을 0, 1, 2로 매핑 (라벨을 학습 시에 변경했으므로)
        if label == -1:
            label = 0
        elif label == 0:
            label = 1
        elif label == 1:
            label = 2

        encoding = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 평가용 텍스트와 라벨 리스트 준비
eval_texts = eval_df['full_text'].tolist()  # 평가 데이터의 텍스트 컬럼
eval_labels = eval_df['positivity'].tolist()  # 평가 데이터의 라벨 컬럼

# 평가 데이터셋 및 DataLoader 생성
eval_dataset = TextDataset(eval_texts, eval_labels, tokenizer, max_len=128)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)


In [27]:
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support


def evaluate_model(model, eval_loader, device):
    model.eval()  # 모델을 평가 모드로 전환
    all_preds = []
    all_labels = []

    with torch.no_grad():
        # tqdm 적용
        for batch in tqdm(eval_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 모델 예측
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 라벨을 원래 값 (-1, 0, 1)으로 복원
    all_preds = [p - 1 for p in all_preds]
    all_labels = [l - 1 for l in all_labels]

    # 평가지표 계산
    accuracy = accuracy_score(all_labels, all_preds)

    # 평가지표 계산
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=1)


    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

# 평가 실행
evaluate_model(model, eval_loader, device)


Evaluating: 100%|██████████| 655/655 [12:51<00:00,  1.18s/it]

Accuracy: 0.4390
Precision: 0.1928
Recall: 0.4390
F1-Score: 0.2679



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
